# Cerium oxidation state determination
In this notebook, the oxidation state of a **simulated** dataset is determined from the fine structure of the cerium M edge. The oxidation state is determined by performing a least squares fit of the data with a background and the two reference edges of Ce$^{3+}$ and Ce$^{4+}$. The reference edges are determined from the dataset itself. This can only be done when pure regions of both states are identified. If this is not available, proper references should be measured and can be used as input. In this case proper care needs to be taken for the experimental parameters such as acceleration voltage, collection angle, thickness, *etc.*


In [1]:
%matplotlib qt

In [2]:
import pyEELSMODEL.api as em
import os 
import numpy as np
import matplotlib.pyplot as plt
from pyEELSMODEL.components.fixedpattern import FixedPattern
from pyEELSMODEL.components.linear_background import LinearBG
from pyEELSMODEL.components.MScatter.mscatterfft import MscatterFFT

In [3]:
#the current directory is important to find the cerium references
cdir = os.getcwd()
print('Current directory is: ' + cdir)

Current directory is: C:\Users\DJannis\PycharmProjects\project\pyEELSMODEL\examples


In [4]:
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask

### Simulation
The EELS map is simulated using two reference spectra of cerium together with a background. The background is a powerlaw with r=3 convolved with the low loss spectrum. The content of Ce$^{3+}$ and Ce$^{4+}$, and t/&lambda; is varied accros the map. Two regions are also identified to only have Ce$^{3+}$ or Ce$^{4+}$ which then will be used as references for furhter processing. *This part is not very important since the outcome will be used as input for the quantification.*

In [5]:
xsize = 128
ysize = 128
maps = np.zeros((2,xsize,ysize))


mask0 =make_rectangular_mask(5, 5, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(90, 90, 20, 30, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 20, 30, (xsize,ysize))
mask3 = make_circular_mask(xsize//2, ysize//2, 0, 20, (xsize,ysize))

maps[0] = 1*mask0  + 0.5*mask2 + 0.7*mask3#ce 3+
maps[1] = 1*mask1 + 0.5*mask2 + 0.3*mask3#ce 4+


In [6]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(maps[0])
ax[0].set_title(r'Ce3+')
ax[1].imshow(maps[1])
ax[1].set_title(r'Ce4+')

Text(0.5, 1.0, 'Ce4+')

Loading of simulated reference cerium M edges. Note that these references replicate experimental results but are quantitatively not correct.  

In [7]:
file_ce3 = os.path.join(cdir, r'data\ce3_edge.hdf5')
ce3 = em.Spectrum.load(file_ce3)
file_ce4 = os.path.join(cdir, r'data\ce4_edge.hdf5')
ce4 = em.Spectrum.load(file_ce4)

In [8]:
fig, ax = plt.subplots()
ax.plot(ce3.energy_axis, ce3.data, label='Ce+3')
ax.plot(ce4.energy_axis, ce4.data, label='Ce+4')
ax.legend()

In [9]:
cte=1
tlambda_map = np.ones_like(mask0)*0.3*cte
tlambda_map[mask0==1] = 0.2
tlambda_map[mask1==1] = 0.3
tlambda_map[mask2==1] = 0.5
tlambda_map[mask3==1] = 0.4

In [10]:
settings = (300e3, 1e-9, 20e-3) #E0, convergence angle, collection angle
msh = em.MultiSpectrumshape(0.05, 840, 4096, xsize, ysize)
sh = msh.getspectrumshape()

sim = em.CoreLossSimulator(msh, [], [], maps, tlambda_map, settings)
sim.fwhm=0.3 #eV, resolution needs to be good enough to resolve difference between fine structure
sim.n_plasmon = 3 #number of plasmons 
sim.make_lowloss()

sim.element_components = []
sim.element_components.append(FixedPattern(sh, ce3))
sim.element_components.append(FixedPattern(sh, ce4))

sim.make_coreloss()


16384it [00:14, 1146.06it/s]
16384it [01:13, 222.49it/s]


In [11]:
hl = sim.multispectrum
ll = sim.ll

### Extract Core-loss edges
In this section, the reference edges are extracted from the simulated dataset. This will be done by first identifying the regions in which a pure oxidation state occurs. Next, a fit will be performed on this averaged edge. The fit includes the background, core-loss edge, fine structure and thickness. From the fit a raw experimental reference edge can be extracted which will be used for the oxidation state determination. Here we used model-based fitting to extract a reference edge which is different from other approaches where the experimental edge after power-law background subtraction gets deconvolved.  

In [13]:
from pyEELSMODEL.operators.quantification.extract_experimental_edge import ExperimentalEdgeExtractor

In [14]:
exp = ExperimentalEdgeExtractor(hl, settings, ll=ll)

Running of the next cell opens a figure on which you can draw a shape with *max_points (4)* corners. Each point is selected by using the right mouse click. After the area is drawn, the figure should be closed. When "the shape is drawn, this will be used to determine the average spectrum
 is printed then you know that the area is configured properly. "\
Two areas need to be selected which is done by running the .define_new_region() function twice. The two regions of pure Ce$^{3+}$ and Ce$^{4+}$ are the rectangular regions.Using the .show_regions(), you can see which areas you have selected. 

In [15]:
exp.define_new_region(max_points=4)

the shape is drawn, this will be used to determine the average spectrum


In [16]:
exp.define_new_region(max_points=4)

the shape is drawn, this will be used to determine the average spectrum


In [17]:
exp.show_regions()
exp.calculate_spectra()

Since we know where the two regions are located, we can also use this as input into the .define_new_region(). This removes the graphical user input which makes analysis less reproducable. The coordinates of the points from which to draw the area should be provided. 

In [ ]:
exp = ExperimentalEdgeExtractor(hl, settings, ll=ll)
exp.define_new_region(max_points=4, coords = [[5,5,25,25],[5,25,25,5]])
exp.define_new_region(max_points=4, coords = [[90,90,120,120],[90,110,110,90]])

In [ ]:
exp.show_regions() #check the proper inputs of the coordinates
exp.calculate_spectra()

Show the average spectra for checking that the proper regions are determined. 

In [18]:
exp.show_average_spectra()

From the average experimental datasets, the edge shape is extracted by model-based fitting this spectrum. In order to do this, one needs to specify the elements and edges which are present in the spectrum. The intervals indicates the energy region over which to fit the fine structure and pre_fine indicates how many eV before the tabulated edge onset energy, the fine structure should already be used. This could be due to a chemical shift or a bad energy axis (wrong offset or dispersion). For more information see description of the function in the docstrings

In [22]:
#to show the docstrings of the function. 
exp.extract_edge?

Signature: exp.extract_edge(index, elements, edges, intervals, pre_fine)
Docstring:
Fits the average spectrum indicated by index using the elements and
edges. The interval indicates the energy region over which to fit
the fine structure. Pre_fine is used to modify the start position
of the fine structure with respect to the atomic cross section. The
sampling of the fine structure is determined by the fwhm of the zlp.

Parameters
----------
index: int
    The index of the chosen area. The first area selected has index 0
    and so forth. The show_regions() function also shows which area
    corresponds to which index.
elements: list
    List of elements which are present in the spectrum of index
edges: list
    List of edge of the elements. The length of this list should be
    equal to the elements list.
intervals: list
    List of energy intervals [eV] over which to fit the fine structure.
pre_fine: list
    List of energies onset energy with respect to the onset energy
    of the ato

In [19]:
fixs0 = exp.extract_edge(index=0, elements=['Ce'], edges=['M'], intervals=[35], pre_fine=[5])
fixs0[0].setname('Ce 3+')

cannot use analytical gradients since a convolutor is inside model


0.29999999999999893
116
True
here


In [20]:
fixs1 = exp.extract_edge(index=1, elements=['Ce'], edges=['M'], intervals=[35], pre_fine=[5])
fixs1[0].setname('Ce 4+')

cannot use analytical gradients since a convolutor is inside model


0.29999999999999893
116
True
here


Compare the inputted spectra with the extracted ones. There is not a perfect match since the background function has a slight bias because the exact functional form of the background is unkown. This means that the result will also be biased because we did not extract the exact physical edge. This also happens in reality where the functional form of the background is also unkown. 

In [21]:
fig, ax = plt.subplots()
ax.plot(fixs0[0].energy_axis, fixs0[0].data/fixs0[0].integrate((950,1000)), label='Ce3+', color='blue')
ax.plot(fixs1[0].energy_axis, fixs1[0].data/fixs1[0].integrate((950,1000)), label='Ce4+', color='red')

ax.plot(ce3.energy_axis, ce3.data/ce3.integrate((950,1000)), color='blue', linestyle='dotted', label='Simulted Ce3+')
ax.plot(ce4.energy_axis, ce4.data/ce4.integrate((950,1000)), color='red', linestyle='dotted', label='Simulted Ce4+')
ax.legend()

### Linear Least Squares Fitting
The reference edges will be used to determine relative oxidation state of the cerium. The model consist out of:
1. Background: Sum of powerlaws (to make it linear)
2. Reference edges for Ce$^{3+}$ and Ce$^{4+}$
3. Low loss to take multiple scattering into account since the t/&lambda; varies over the sample 

#### Making the model

In [23]:
from pyEELSMODEL.components.linear_background import LinearBG
from pyEELSMODEL.components.MScatter.mscatterfft import MscatterFFT

In [24]:
n = 4
bg = LinearBG(specshape=hl.get_spectrumshape(), rlist=np.linspace(1,5,n))

In [25]:
refs = [fixs0[0], fixs1[0]] #list of the reference edges

In [26]:
#the reference should be a FixedPattern object. 
refs[0]

In [27]:
llcomp  = MscatterFFT(hl.get_spectrumshape(), ll)

In [28]:
components = [bg]+refs+[llcomp]
mod = em.Model(hl.get_spectrumshape(), components)

#### Fitting the experimental data

In [29]:
fit = em.LinearFitter(hl, mod, use_weights=True)
fit.multi_fit()

cannot use analytical gradients since a convolutor is inside model
16384it [00:28, 576.06it/s]


In [30]:
fig, exp_maps, names = fit.show_map_result(refs)

In [32]:

sum_ce = (exp_maps[0]+exp_maps[1])

con_ce3 = exp_maps[0]/sum_ce #relative ratio ce3 from experimental data
con_ce4 = exp_maps[1]/sum_ce #relative ratio ce4 from experimental data

mask = sum_ce < 1e-3 #identify when no cerium is detected
con_ce3[mask] = np.nan
con_ce4[mask] = np.nan

sim_ce3 = maps[0]/(maps[0]+maps[1]) #ground truth
sim_ce4 = maps[1]/(maps[0]+maps[1]) #ground truth


C:\Users\DJannis\AppData\Local\Temp\ipykernel_25264\1042502816.py:10: RuntimeWarning: invalid value encountered in divide
  sim_ce3 = maps[0]/(maps[0]+maps[1]) #ground truth
C:\Users\DJannis\AppData\Local\Temp\ipykernel_25264\1042502816.py:11: RuntimeWarning: invalid value encountered in divide
  sim_ce4 = maps[1]/(maps[0]+maps[1]) #ground truth


Compare the fitted ration with the ground truth. The comparison seems to be quite good even though there is a bias from the background.

In [33]:
fig, ax = plt.subplots(2,2)
cmp = 'jet'
ax[0,0].imshow(con_ce3,vmin=0,vmax=1, cmap=cmp)
ax[0,0].set_title(r'Fitted ratio')
ax[0,0].set_ylabel(r'Ce 3+')
ax[0,1].imshow(sim_ce3,vmin=0,vmax=1, cmap=cmp)
ax[0,1].set_title(r'Ground truth ratio')
ax[1,0].imshow(con_ce4,vmin=0,vmax=1, cmap=cmp)
ax[1,0].set_ylabel(r'Ce4+')
ax[1,1].imshow(sim_ce4,vmin=0,vmax=1, cmap=cmp)


One can visualze the fitted multispectra

In [34]:
multimodels = []
multimodels.append(fit.model_to_multispectrum_with_comps([bg])) #only background
multimodels.append(fit.model_to_multispectrum_with_comps([bg, refs[0]])) #background and ce3+
multimodels.append(fit.model_to_multispectrum_with_comps([bg, refs[1]])) #background and ce4+
multimodels.append(fit.model_to_multispectrum_with_comps([bg, refs[0], refs[1]])) #total model


16384it [00:17, 946.33it/s]
16384it [00:17, 948.46it/s]
16384it [00:18, 905.17it/s]
16384it [00:17, 962.24it/s]


In [35]:
em.MultiSpectrumVisualizer([hl]+multimodels, labels=['Exp', 'Bg', 'Ce3', 'Ce4', 'full'])